In [1]:
import gensim
import pandas as pd
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re
import pyLDAvis
import pyLDAvis.gensim_models


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/diegojimenez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/diegojimenez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/diegojimenez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df=pd.read_csv("/Users/diegojimenez/Documents/GitHub/computational_ds/data/output/updated_dataframe_with_clusters_word2vec.csv")
df.head(5)

,ID,Headline,Date,Text,Organization,Link,Cluster
0,1,China says it’s building new homegrown aircraf...,"Updated 1:09 PM EST, Fri January 1, 2016",Story highlights The U.S. military has long be...,CNN,https://edition.cnn.com/2015/12/31/asia/china-...,619
1,2,"Hillary Clinton emails: Kissinger, Photoshop a...","Updated 9:34 PM EST, Thu December 31, 2015",Story highlights The State Department released...,CNN,https://edition.cnn.com/2015/12/31/politics/cl...,-1
2,3,How the stars rang in 2016,"Published 9:04 AM EST, Fri January 1, 2016",Story highlights Some stars hit exotic locatio...,CNN,https://edition.cnn.com/2016/01/01/entertainme...,-1
3,4,Smoke still wafting from Dubai’s luxury Addres...,"Updated 4:55 PM EST, Fri January 1, 2016",Story highlights NEW: Fire has been contained ...,CNN,https://edition.cnn.com/2016/01/01/middleeast/...,482
4,5,Super PACs backing Cruz to launch TV ad blitz,"Published 8:36 PM EST, Thu December 31, 2015",Story highlights Ads supporting Cruz set to ai...,CNN,https://edition.cnn.com/2015/12/31/politics/cr...,1428


In [4]:
len(df)

446967

In [5]:
def preprocess_text(df, text_column, custom_stopwords):
    clean_text_column = f"{text_column}_cleaned"
    lemmatizer = WordNetLemmatizer()

    def clean_single_text(text):
        if not isinstance(text, str):
            return []

        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in custom_stopwords]
        tokens = [token for token in tokens if token.isalpha()]
        tokens = [token for token in tokens if not token.isdigit()]
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        tokens = [token for token in tokens if token not in stop_words]

        return tokens

    # Apply the cleaning function to the specified column
    df[clean_text_column] = df[text_column].apply(clean_single_text)

    return df

# Define your custom stopwords and stop words
custom_stopwords = ["reuters", "fox", "cnn", "was", "has", "us", "year"]
stop_words = set(stopwords.words('english'))

# Call the function and update the DataFrame
df_final = preprocess_text(df, "Text", custom_stopwords)

# Save the DataFrame to a CSV file
df_final.to_csv('cleaned_dataframe.csv', index=False, encoding='utf-8')

print("Cleaned text has been added to the DataFrame and saved to 'cleaned_dataframe.csv'")

Cleaned text has been added to the DataFrame and saved to 'cleaned_dataframe.csv'


In [3]:
df_final=pd.read_csv("/Users/diegojimenez/Documents/GitHub/computational_ds/src/topic_modeling/cleaned_dataframe.csv")
df_final["Text_cleaned"] = df_final["Text_cleaned"].apply(lambda x: eval(x))

In [6]:
df_final.head(5)

,ID,Headline,Date,Text,Organization,Link,Cluster,Text_cleaned
0,1,China says it’s building new homegrown aircraf...,"Updated 1:09 PM EST, Fri January 1, 2016",Story highlights The U.S. military has long be...,CNN,https://edition.cnn.com/2015/12/31/asia/china-...,619,"[story, highlight, military, long, believed, c..."
1,2,"Hillary Clinton emails: Kissinger, Photoshop a...","Updated 9:34 PM EST, Thu December 31, 2015",Story highlights The State Department released...,CNN,https://edition.cnn.com/2015/12/31/politics/cl...,-1,"[story, highlight, state, department, released..."
2,3,How the stars rang in 2016,"Published 9:04 AM EST, Fri January 1, 2016",Story highlights Some stars hit exotic locatio...,CNN,https://edition.cnn.com/2016/01/01/entertainme...,-1,"[story, highlight, star, hit, exotic, location..."
3,4,Smoke still wafting from Dubai’s luxury Addres...,"Updated 4:55 PM EST, Fri January 1, 2016",Story highlights NEW: Fire has been contained ...,CNN,https://edition.cnn.com/2016/01/01/middleeast/...,482,"[story, highlight, new, fire, contained, guest..."
4,5,Super PACs backing Cruz to launch TV ad blitz,"Published 8:36 PM EST, Thu December 31, 2015",Story highlights Ads supporting Cruz set to ai...,CNN,https://edition.cnn.com/2015/12/31/politics/cr...,1428,"[story, highlight, ad, supporting, cruz, set, ..."


In [8]:
df_final["Text_cleaned"] = df_final["Text_cleaned"].apply(lambda x: eval(x))

In [5]:
# Create a dictionary from the documents
dictionary = corpora.Dictionary(df_final["Text_cleaned"])

# Create a corpus from the documents
corpus = [dictionary.doc2bow(doc) for doc in df_final["Text_cleaned"]]
print(len(corpus))

446967


In [6]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=2)

In [7]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

pyLDAvis.save_html(vis, 'lda_visualization.html')


In [ ]:
#lda_model_2 = LdaModel(corpus, num_topics=8, id2word=dictionary, passes=5)

In [ ]:
#pyLDAvis.enable_notebook()
#vis_2 = pyLDAvis.gensim_models.prepare(lda_model_2, corpus, dictionary)
#pyLDAvis.display(vis_2)

In [ ]:
#pyLDAvis.enable_notebook()
#vis_3 = pyLDAvis.gensim_models.prepare(lda_model_3, corpus, dictionary)
#pyLDAvis.display(vis_2)

In [ ]:
#lda_model_5 = LdaModel(corpus, num_topics=2000, id2word=dictionary, passes=5)